
# Table of Contents



This begins a series of `org-mode` notebooks with time-efficient solutions to
the [Project Euler](https://projecteuler.net/) problems. I didn't take an algorithms class for my computer
science degree, but I love math, so I figured this would be a fine way to learn
those skills. Additionally, I need some well-structured problems to learn the
ins and outs of `org-mode`, and this might as well be it. I plan to use Python
throughout, leveraging `numpy` for its speed with linear algebra, and the
`%timeit` python magic to compare run times between solutions.

Initialize Python session with imports.



In [1]:
from array import array
from functools import reduce
import numpy as np
from math import log
from operator import mul

## 1 Multiples of 3 and 5



If we list all the natural numbers below 10 that are multiples of 3 or 5,
we get 3, 5, 6 and 9. The sum of these multiples is 23.

Find the sum of all the multiples of 3 or 5 below 1000.



### A:



We can write the solution in terms of triangular numbers, making sure to
subtract all multiples of fifteen since they were double-counted by the first
two terms:

\begin{eqnarray*}
S & = & (3+6+\ldots+999) + (5+10+\ldots+995) - (15+30+\ldots+990) \\
  & = & 3(1+2+\ldots+333) + 5(1+2+\ldots+199) - 15(1+2+\ldots+66) \\
  & = & 3T_{333} + 5T_{199} - 15T_{66}
\end{eqnarray*}

It is quicker to solve it analytically, even if generality is lost. The
triangular number $T_N$ can be found in $O(1)$. Check out the Wikipedia
entry for [triangular numbers](https://en.wikipedia.org/wiki/Triangular_number) if you aren't already familiar with this formula.

$$ T_N = 1+2+\ldots+N = \frac{N(N+1)}{2}. $$



In [1]:
def triangular(n):
    """Find the ``n``th triangular number."""
    return n*(n+1)//2

assert triangular(1)==1
assert triangular(2)==3
assert triangular(100)==5050

The $T_N$ we use for a factor, $f$, is easily found given our upper bound,
$n$. We know $fN\leq n$ and $N$ must be an integer. Therefore, 

$$ N = \lfloor n/f \rfloor $$

The sum of all multiples of $f_1,f_2$ that are less than $n$ is written as
follows:

$$ S_n(f_1,f_2) = f_1\frac{N_1(N_1+1)}{2} + f_2\frac{N_2(N_2+1)}{2} -
f_3\frac{N_3(N_3+1)}{2} $$

where $N_i = \lfloor n/f_i \rfloor$ and $f_3 = f_1f_2$.



In [1]:
def sum_of_multiples(f1,f2,n):
    """Find the sum of multiples of ``f1`` and ``f2`` less than ``n``."""
    n0 = n-1
    f3 = f1*f2
    return f1*triangular(n0//f1)+f2*triangular(n0//f2)-f3*triangular(n0//(f3))

assert sum_of_multiples(3,5,10) == 23

print(sum_of_multiples(3,5,1000))
%timeit sum_of_multiples(3,5,1000)

233168
529 ns ± 8.78 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)

$O(1)$ scaling!



In [1]:
print(sum_of_multiples(3,5,10**9))
%timeit sum_of_multiples(3,5,10**9)

233333333166666668
689 ns ± 4.62 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)

There is a way to generalize this for list of factors, but that is far beyond
the scope of this problem.



## 2 Even Fibonacci numbers



Each new term in the Fibonacci sequence is generated by adding the previous
two terms. By starting with 1 and 2, the first 10 terms will be:
1, 2, 3, 5, 8, 13, 21, 34, 55, 89, &#x2026;

By considering the terms in the Fibonacci sequence whose values do not exceed
four million, find the sum of the even-valued terms.



### A:



Briefly ignoring that the problem statement contains the wrong starting numbers
for the Fibonacci sequence, this is quick and easy to solve by brute force since
there aren't too many Fibonacci numbers below 4000000.



In [1]:
def fibonacci_sequence(n,init=[1,1]):
    """Generate a sequence of Fibonacci numbers, ``F_n<n``."""
    if n<1:
        return None
    F = init
    F_next = F[-2]+F[-1]
    while F_next < n:
        F += [F_next]
        F_next = F[-2]+F[-1]
    return F

assert fibonacci_sequence(2) == [1,1]
assert fibonacci_sequence(90,[1,2]) == [1,2,3,5,8,13,21,34,55,89]

Some marginal savings can be made by recognizing that only $F_{3i+2}$ are even
and using list slicing to extract those numbers quickly, rather than checking if
every number is even. See the table below as clarification for why this pattern
arises.

$$ \text{odd} + \text{even} = \text{odd} $$

$$ \text{odd} + \text{odd} = \text{even} $$

| i|1|2|3|4|5|6|7|8|&#x2026;|3i|3i+1|3i+2|
|---|---|---|---|---|---|---|---|---|---|---|---|---|
| $parity(F_i)$|odd|even|odd|odd|even|odd|odd|even|&#x2026;|odd|odd|even|



In [1]:
seq = fibonacci_sequence(4000000,[1,2])
print(sum(seq[1::3]))

%timeit sum(fibonacci_sequence(4000000,[1,2])[1::3])

4613732
3.66 µs ± 38.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

## 3 Largest prime factor



The prime factors of 13195 are 5, 7, 13 and 29.

What is the largest prime factor of the number 600851475143?



### A:



This can be solved by prime factorization, keeping in mind that all primes
above two are odd and indivisible by three. In the code below we don't bother to
save the exponents since we only really care about the factors. Additionally, we
save a huge amount of time by breaking the loop as soon as the placeholder $n$
is guaranteed to be prime or 1.



In [1]:
def prime_factors(n):
    """Find the prime factors of ``n``. Returns only primes, not their
    exponents.
    """
    p = []
    for f in [2,3]:
        if n%f==0:
            p += [f]
            while n%f==0:
                n //= f
    f = 5
    while f*f <= n:
        if n%f==0:
            p += [f]
            while n%f==0:
                n //= f
        f += 2 if f%6>1 else 4
    if n > 1:
        p += [n]
    return p

assert prime_factors(1) == []
assert prime_factors(2) == [2]
assert prime_factors(4) == [2]
assert prime_factors(6) == [2,3]
assert prime_factors(24) == [2,3]
assert prime_factors(25) == [5]

For large numbers and no supplied list of primes the run time of the algorithm
is $O(n)$.



In [1]:
print(prime_factors(600851475143)[-1])
%timeit prime_factors(600851475143)[-1]

6857
73.2 µs ± 6.02 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)

## 4 Largest palindrome product



A palindromic number reads the same both ways. The largest palindrome made
from the product of two 2-digit numbers is 9009 = 91 × 99.

Find the largest palindrome made from the product of two 3-digit numbers.



### A:



Starting from the upper bound will require the fewest iterations. First we need
a function to construct a palindrome from a prefix:



In [1]:
def make_palindrome(prefix):
    """Find a palindromic number from a prefix."""
    pre = str(prefix)
    suf = "".join(reversed(pre))
    pal = pre+suf
    return int(pal)

assert make_palindrome(999)==999999

Then, for each possible palindrome, we check for integer products and terminate
once they are found.



In [1]:
def largest_palindrome_product(k):
    """Finds the largest palindrome that is a product of two ``k``-digit
    numbers.
    """
    isfound = False
    pre = 10**k-1
    while not isfound:
        pal = make_palindrome(pre)
        fmin = 10**k-1
        for f in range(fmin,10**(k-1)-1,-1):
            if (pal/f)>fmin or f*f<pal:
                break
            if pal%f==0:
                isfound = True
                fact = {f, pal//f}
                break
        pre += -1
    return pal, fact

assert largest_palindrome_product(2)==(9009,{91,99})

print(largest_palindrome_product(3))
%timeit largest_palindrome_product(3)

(906609, {993, 913})
451 µs ± 5.06 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

## 5 Smallest multiple



2520 is the smallest number that can be divided by each of the numbers from
1 to 10 without any remainder.

What is the smallest positive number that is evenly divisible by all of the
numbers from 1 to 20?



### A:



Consider these two composite numbers and their forms in prime factorization.

$$ 12 = 2^2\cdot3^1\cdot5^0 \\
   15 = 2^0\cdot3^1\cdot5^1 $$

We know that $lcm(12,15)=60=2^2\cdot3^1\cdot5^1$. In other words, the prime
factorization gives us a way to quickly find the LCM. In general terms this
means

\begin{eqnarray*}
k_a          & = & \prod_i p_i^{a_i} \\
k_b          & = & \prod_i p_i^{b_i} \\ 
lcm(k_a,k_b) & = & \prod_i p_i^{\max\{a_i,b_i\}}
\end{eqnarray*}

For all $k\leq n\in\mathbb{N}$, the LCM must be the product of the primes
raised to their maximum possible power within the limit given. I used [the Sieve
of Eratosthenes](https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes) to get a list of primes below an upper bound. You should check
out the Wiki page for a cool visualization of the algorithm. I use `array` data
structures instead of preallocated lists. `array` is a light wrapper on the C
`array`.



In [1]:
def esieve(n):
    """Find all primes less than ``n`` with the Sieve of Eratosthenes."""
    if n<=2:
        return array('l',[])

    n_odd = n//2
    isprime = array('b',[True]*n_odd)
    for k in range(3,int(n**0.5)+1,2):
        if isprime[k//2]:
            for i in range(k*k//2,n_odd,k):
                isprime[i] = False

    primes = array('l',[2])
    for i in range(1,n//2):
        if isprime[i]:
            primes.append(2*i+1)
    return primes

assert len(esieve(2)) == 0
assert esieve(3) == array('l',[2])
assert esieve(10) == array('l',[2,3,5,7])
assert esieve(20) == array('l',[2,3,5,7,11,13,17,19])

For the sequence we were given, every natural number up to a limit $n$ is
represented. For every prime $p\leq n$, there is a maximum possible exponent
$a$ such that

$$ p^a \leq n $$

Solving for $a$ gives

$$ a = \lfloor \log_p n \rfloor = \lfloor \ln n / \ln p \rfloor $$

Therefore, the least common multiple of the series $1,2,...,n$ is the product

$$ lcm(1,2,\ldots,n) = p_1^{a_1}p_2^{a_2}\ldots p_k^{a_k} $$

where $p_k\leq n$ are prime and $a_k = \lfloor \log_p n \rfloor$.



In [1]:
def lcm_naturals(n,primes=None):
    """Find the smallest common multiple of the sequence of natural numbers
    `i<=n'.
    """
    if primes is None:
        primes = esieve(n+1)
    return reduce(mul, [1] + [p**int(log(n)/log(p)) for p in primes])

assert lcm_naturals(10) == 2520

print(lcm_naturals(20))
%timeit lcm_naturals(20)

232792560
6.89 µs ± 47 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

## 6 Sum square difference



The sum of the squares of the first ten natural numbers is,

$$ 1^2 + 2^2 + \ldots + 10^2 = 385 $$

The square of the sum of the first ten natural numbers is,

$$ (1 + 2 + \ldots + 10)^2 = 55^2 = 3025 $$

Hence the difference between the sum of the squares of the first ten natural
numbers and the square of the sum is 3025 − 385 = 2640.

Find the difference between the square of the sum and the sum of the squares of
the first one hundred natural numbers.



### A:



Recall from problem 1 the formula for the $n$th triangular number, $T_n$.

$$ T_n = 1+2+\ldots+n = \frac{n(n+1)}{2} $$

Now consider the [square pyramidal numbers](https://en.wikipedia.org/wiki/Square_pyramidal_number), $P_n$. Again, if you aren't
familiar with this series or its derivation, check out the Wikipedia page.

$$ P_n = 1^2+2^2+\ldots+n^2 = \frac{n(n+1)(2n+1)}{6} $$



In [1]:
def square_pyramidal(n):
    """Find the ``n``th square pyramidal number."""
    return n*(n+1)*(2*n+1)//6

assert square_pyramidal(1)==1
assert square_pyramidal(2)==5
assert square_pyramidal(4)==30

Let $f$ be the difference between the sum of the squares and the square of the
sum for the first $n$ natural numbers. 

\begin{eqnarray*}
f(n) & = & \left[\sum_{i=1}^n i \right]^2 -
           \left[\sum_{i=1}^n i^2 \right] \\
     & = & T_n^2 - P_n
\end{eqnarray*}



In [1]:
def sum_sq_diff(n):
    """For the first ``n`` natural numbers, find the difference between the
    square of the sum and the sum of the squares.
    """
    return triangular(n)**2 - square_pyramidal(n)

assert sum_sq_diff(10) == 2640

print(sum_sq_diff(100))
%timeit sum_sq_diff(100)

25164150
522 ns ± 7.52 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)

## 7 10001st prime



By listing the first six prime numbers: 2, 3, 5, 7, 11, and 13, we can see
that the 6th prime is 13.

What is the 10 001st prime number?



### A:



We should use the Sieve of Eratosthenes (see problem 5) to find this, but first
we need to approximate what our upper bound should be. [According to Dusart](https://en.wikipedia.org/wiki/Prime_number_theorem#Approximations_for_the_nth_prime_number), the
$n$th prime, $p_n$, is bound by the following inequality.

$$ n(\log n + \log\log n - 1) < p_n < n(\log n + \log\log n) $$

for $n\geq6$.



In [1]:
def prime_bounds(n):
    """Find the lower and upper bounds of the ``n``th prime."""
    if n<6:
        return [1,14]
    else:
        lim = log(n)+log(log(n))
        return [int(n*(lim-1)),int(n*lim+1)]

assert prime_bounds(4) == [1,14]
assert prime_bounds(6) == [8,15]
assert prime_bounds(10) == [21,32]

I used the upper bound from the above calculation to perform a Sieve of
Eratosthenes and return the 10001st prime. It is interesting to note how
close the prime is to the lower bound.

The overall algorithm should be about the same complexity as the sieve,
$O(n\log\log n)$. Any additional complexity results from systematic
overestimation of the upper bound.



In [1]:
bounds = prime_bounds(10001)
print(bounds)
print(esieve(bounds[1])[10000])

%timeit esieve(prime_bounds(10001)[1])[10000]

[104318, 114320]
104743
10.1 ms ± 36.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

## 8 Largest product in a series



The four adjacent digits in the 1000-digit number that have the greatest
product are 9 × 9 × 8 × 9 = 5832.



In [1]:
number = "73167176531330624919225119674426574742355349194934\
96983520312774506326239578318016984801869478851843\
85861560789112949495459501737958331952853208805511\
12540698747158523863050715693290963295227443043557\
66896648950445244523161731856403098711121722383113\
62229893423380308135336276614282806444486645238749\
30358907296290491560440772390713810515859307960866\
70172427121883998797908792274921901699720888093776\
65727333001053367881220235421809751254540594752243\
52584907711670556013604839586446706324415722155397\
53697817977846174064955149290862569321978468622482\
83972241375657056057490261407972968652414535100474\
82166370484403199890008895243450658541227588666881\
16427171479924442928230863465674813919123162824586\
17866458359124566529476545682848912883142607690042\
24219022671055626321111109370544217506941658960408\
07198403850962455444362981230987879927244284909188\
84580156166097919133875499200524063689912560717606\
05886116467109405077541002256983155200055935729725\
71636269561882670428252483600823257530420752963450"

v = np.array([int(d) for d in number],dtype=int)

Find the thirteen adjacent digits in the 1000-digit number that have the
greatest product. What is the value of this product?



### A:



This is a moving window problem. Let $N$ be a number with $n$ digits,  $\mathbf{p}$
be a vector where the elements $p_i$ are the products of $w > 1$ adjacent
digits of $N$, starting with $d_i$.

$$ N = d_1d_2...d_n $$

$$ \mathbf{p} \in \mathbb{R}^{n-w+1} $$

$$ p_i = \prod_{k=i}^{i+w-1} d_k $$



In [1]:
def adjacent_products(v,w):
    """Find the products of `w' adjacent elements in the vector `v'."""
    n = len(v)
    p = np.ones((n-w+1,),dtype=int)
    for i in range(0,w):
        p = np.multiply(p,v[i:n+i-w+1])
    return p

assert max(adjacent_products(v,4)) == 5832

`numpy` does vector operations extremely quickly.



In [1]:
print(max(adjacent_products(v,13)))
%timeit adjacent_products(v,13)

23514624000
20.7 µs ± 88.3 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)

## 9 Special Pythagorean triplet



A Pythagorean triple is a set of three natural numbers, $a < b < c$, for
which,

$$ a^2 + b^2 = c^2 $$

For example, $3^2 + 4^2 = 9 + 16 = 25 = 5^2$.

There exists exactly one Pythagorean triplet for which $a + b + c = 1000$.
Find the product $abc$.



### A:



Using [Euclid's formula](https://en.wikipedia.org/wiki/Pythagorean_triple#Generating_a_triple) for the primitive Pythagorean triples:

\begin{eqnarray*}
a & = & m^2-n^2 \\
b & = & 2mn \\
c & = & m^2+n^2
\end{eqnarray*}

where $m>n>0$, $m$ and $n$ are coprime ($gcd(m,n)=1$) and one is even.
Since we don't necessarily need these triples to be primitive, we'll rewrite the
formula by adding in some common divisor, $d$.

\begin{eqnarray*}
a & = & d(m^2-n^2) \\
b & = & d(2mn) \\
c & = & d(m^2+n^2)
\end{eqnarray*}

Introducing a constraint on the sum, $a+b+c=s$, gives

\begin{eqnarray*}
s & = & a + b + c \\
  & = & d(m^2-n^2) + d(2mn) + d(m^2+n^2) \\
  & = & 2dm^2 + 2dmn \\
  & = & 2dm(m+n) \\
  & = & 2dmk
\end{eqnarray*}

A few interesting things to point out here. Try plugging in numbers if any of
these points don't make sense.

1.  $s$ must be divisible by 2.
2.  $s/2$ must be divisible by $m$.
3.  $m < \sqrt{s/2}$ because $m < k = m+n$.
4.  $s/(2m)$ must be divisible by $k = m+n$.
5.  $m$ and $k = m+n$ are coprime ($gcd(m,k)=1$) since $m$ and $n$ are coprime.
6.  $k = m+n$ is odd.

From these observations we can write an algorithm that, in the outer loop,
checks all valid $m$, and, in the inner loop, checks all valid $k$ for the
chosen $m$. We know that there is only one triple for $a+b+c=1000$, but if
we spend the extra time to check all possible entries, we can generalize this
code for problem 39 and 75.



In [1]:
def gcd(a, b):
    """Find the greatest common divisor of two integers."""
    while b != 0:
        (a, b) = (b, a%b)
    return a

def pythagorean_triples_from_sum(s):
    """Find all Pythagorean triples ``a^2+b^2=c^2`` such that ``a+b+c=s`` and
    ``0<a<b<c<s``.
    """
    if s%2 == 1:
        return []
    trip = []
    for m in range(2,int((s/2)**0.5)+1):
        if (s//2)%m==0:
            if m%2==0:
                k = m+1
            else:
                k = m+2
            while (k < 2*m) and k <= s/(2*m):
                if (s//(2*m))%k==0 and gcd(m,k)==1:
                    d = s//(2*k*m)
                    n = k - m
                    a = d*(m**2-n**2)
                    b = d*2*m*n
                    c = d*(m**2+n**2)
                    trip += [(min(a,b),max(a,b),c)]
                k += 2
    return trip

assert pythagorean_triples_from_sum(12)[0]==(3,4,5)
assert pythagorean_triples_from_sum(40)[0]==(8,15,17)

The algorithm is extremely efficient for this small sum.



In [1]:
a,b,c = pythagorean_triples_from_sum(1000)[0]
print([a,b,c])
print(a*b*c)

%timeit pythagorean_triples_from_sum(1000)

[200, 375, 425]
31875000
5.58 µs ± 45.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

It is also scalable to large sums since it only needs to check the primitive
triples.



In [1]:
triples = pythagorean_triples_from_sum(10**9)
for a,b,c in triples:
    print([a,b,c])

%timeit pythagorean_triples_from_sum(10**9)

[200000000, 375000000, 425000000]
[218750000, 360000000, 421250000]
[23437500, 488000000, 488562500]
12.8 ms ± 108 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

## 10 Summation of primes



The sum of the primes below 10 is 2 + 3 + 5 + 7 = 17.

Find the sum of all the primes below two million.



### A:



Sieve of Eratosthenes.



In [1]:
print(sum(esieve(2000000)))
%timeit sum(esieve(2000000))

142913828922
169 ms ± 6.08 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)